# Segmenting and Clustering Neighborhoods in Toront

##### project notebook

In [21]:

import pandas as pd
import numpy as np
import requests
import lxml
import html5lib
from bs4 import BeautifulSoup

#!pip install beautifulsoup4

##### getting the wiki doc and reading it with the beautifulsoup 

In [16]:
htmlR = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
htmlF = htmlR.text
soup = BeautifulSoup(htmlF, 'lxml')
#print(soup.prettify())

##### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood, let's retreive them from the soup 

In [151]:
code = soup.find('div', class_ = 'mw-parser-output') 
tab = code.table.tbody.text  # get the table from the div section 

#convert the tab to a dataframe

tabsplit = tab.split('\n')
tabsplit = [i for i in tabsplit if i!='']

tabsplit = np.array(tabsplit).reshape((-1 , 3))

df = pd.DataFrame(tabsplit[1::,:] , index = None , columns =tabsplit[0 ,:] )

#selection of  only the rows that have Borough 
df = df[df['Borough']!= 'Not assigned' ]
print(df.columns)
df.head()

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### concat neighborhods that have the same postcode 

In [164]:
#concat neighborhods that have the same postcode 
newdf = df.groupby(['Postcode'])['Neighbourhood'].apply(','.join).reset_index()
newdf = newdf.sort_values('Postcode', axis=0)

t = df.drop_duplicates('Postcode').sort_values('Postcode', axis=0).reset_index()
newdf['Borough'] = t['Borough']
newdf.head()

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge,Malvern",Scarborough
1,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough
2,M1E,"Guildwood,Morningside,West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


##### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [172]:
# neighborhood will be the sam 'Not assigned' e as the borough if not assigned
newdf["Neighbourhood"]== 'Not assigned' 
newdf.loc[newdf["Neighbourhood"]== 'Not assigned', 'Neighbourhood'] = newdf['Borough']

#verify by newdf.loc[newdf["Neighbourhood"]== 'Not assigned', 'Neighbourhood']
newdf.head()

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge,Malvern",Scarborough
1,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough
2,M1E,"Guildwood,Morningside,West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


In [173]:
newdf.shape

(103, 3)